# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import difflib

/home/slewie/anaconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased", device_map='cuda')

In [11]:
class SpellChecker:
    def __init__(self, device='cpu'):
        self.device = device
    
    def spell_check(self, sentence):
        sentence_list = sentence.split()
        res = [self._spell_check_at(sentence_list, i) for i in range(len(sentence_list))]
        
        corrected_sentence = []
        for my_dict in res:
            top_keys = sorted(my_dict, key=my_dict.get, reverse=True)[:1][0]
            corrected_sentence.append(top_keys)
            
        return " ".join(corrected_sentence)
    
    def _spell_check_at(self, sentence_list, candidate_ind, topk=150):
        candidate = sentence_list[candidate_ind]
        sentence_list[candidate_ind] = '[MASK]'
        text = " ".join(sentence_list)

        input_ids = tokenizer.encode(text, return_tensors="pt").to(self.device)

        mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

        with torch.no_grad():
            output = model(input_ids)
            logits = output.logits

        mask_logits = logits[0, mask_token_index, :]

        probabilities = torch.softmax(mask_logits, dim=-1)

        top_k_probs, top_k_tokens = torch.topk(probabilities, k=topk)
        top_k_probs = top_k_probs[0]
        top_k_tokens = top_k_tokens[0]
        
        top_k_words = [tokenizer.decode([token_id]) for token_id in top_k_tokens]

        output = {}
        for word, prob in zip(top_k_words, top_k_probs):
            score = difflib.SequenceMatcher(None, word, candidate).ratio() + prob 
            output[word] = score
        sentence_list[candidate_ind] = candidate
        return output

In [12]:
spell_checker = SpellChecker('cuda')

In [13]:
res = spell_checker.spell_check('I love my kat and my dogg')

In [14]:
res

'i love my cat and my dog'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [ ]:
# Your code here